Placing Orders and Executing Trades

In [19]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta, timezone
import json

In [21]:
# start_time = datetime.now(timezone.utc) - timedelta(days = 5)
# end_time = datetime.now(timezone.utc)

In [23]:
df = pd.read_csv('oanda.cfg')

In [46]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, config_file, instrument, bar_length, window, units):
        super().__init__(config_file)
        self.instrument = instrument
        self.bar_length = bar_length
        self.tick_data = pd.DataFrame()
        self.raw_data = pd.DataFrame()
        self.position = 0
        self.units = units
        self.window = window

        try:
            account_summary = self.get_account_summary()
            print("Account Summary:", account_summary)
        except Exception as e:
            print("Error retrieving account summary:", e)

        # Call the method to fetch historical data
        self.get_historical_data()

    def get_historical_data(self):
        """Fetch historical data from OANDA with proper error handling."""
        # Get the current time in UTC
        end_time = datetime.now(timezone.utc)
        start_time = end_time - timedelta(minutes=30)  # Reduced to 30 minutes for testing

        # Format timestamps according to OANDA's requirements
        # Remove milliseconds and timezone info
        start_str = start_time.strftime('%Y-%m-%dT%H:%M:%S.000000Z')
        end_str = end_time.strftime('%Y-%m-%dT%H:%M:%S.000000Z')

        try:
            # Try different parameter combinations
            param_combinations = [
                {'price': 'M', 'granularity': 'M1'},  # Mid price, 1-minute
                {'price': 'B', 'granularity': 'M1'},  # Bid price, 1-minute
                {'price': 'A', 'granularity': 'M1'},  # Ask price, 1-minute
                {'price': 'MBA', 'granularity': 'M1'} # Mid/Bid/Ask prices, 1-minute
            ]

            for params in param_combinations:
                print(f"\nTrying parameters: {params}")
                try:
                    df = self.get_history(
                        instrument=self.instrument,
                        start=start_str,
                        end=end_str,
                        granularity=params['granularity'],
                        price=params['price']
                    )
                    
                    if df is not None and not df.empty:
                        self.tick_data = df
                        print(f"Successfully fetched data with parameters: {params}")
                        print("\nFirst few rows of raw data:")
                        print(df.head())
                        self.process_data()
                        return  # Exit if successful
                    
                except Exception as e:
                    print(f"Failed with parameters {params}: {str(e)}")
                    continue

            # If we get here, none of the combinations worked
            print("\nAll parameter combinations failed. Trying to get available instruments...")
            
            try:
                # Try to get list of available instruments
                accounts = self.get_accounts()
                print("\nAvailable accounts:", accounts)
                
                # Try to get account details
                account_details = self.get_account_summary()
                print("\nAccount details:", account_details)
                
            except Exception as e:
                print(f"Error getting account information: {str(e)}")

        except Exception as e:
            print(f"Error fetching historical data: {str(e)}")
            print("\nPlease verify:")
            print("1. Your API credentials are correct")
            print("2. The instrument symbol is valid (e.g., 'EUR_USD')")
            print("3. Your account has permissions to access this instrument")
            print("4. The requested time range is valid")

    def process_data(self):
        """Process the fetched historical data."""
        try:
            if self.tick_data.empty:
                print("No data to process")
                return

            # Create a copy of the data for processing
            processed_data = self.tick_data.copy()

            # Rename the close price column if it exists
            if 'c' in processed_data.columns:
                processed_data.rename(columns={'c': self.instrument}, inplace=True)

            # Resample the data according to the specified bar length
            processed_data = processed_data.resample(
                self.bar_length, 
                label="right"
            ).last().dropna()

            self.tick_data = processed_data
            
            print(f"\nProcessed Data Summary:")
            print(f"Number of bars: {len(processed_data)}")
            if not processed_data.empty:
                print(f"Date range: {processed_data.index.min()} to {processed_data.index.max()}")
                print("\nFirst few rows of processed data:")
                print(processed_data.head())

        except Exception as e:
            print(f"Error processing data: {str(e)}")

    def __repr__(self):
        return f"ConTrader(instrument='{self.instrument}', bar_length='{self.bar_length}', units={self.units})"

In [48]:
dir(tpqoa.tpqoa)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_stream_data_failsafe_thread',
 'cancel_order',
 'create_order',
 'get_account_summary',
 'get_history',
 'get_instruments',
 'get_positions',
 'get_prices',
 'get_transaction',
 'get_transactions',
 'on_success',
 'print_transactions',
 'retrieve_data',
 'stream_data',
 'stream_data_failsafe',
 'transform_datetime']

In [50]:
trader = ConTrader(
    config_file="oanda.cfg",
    instrument="EUR_USD",
    bar_length="1min",
    window=1,
    units=5000
)

Account Summary: {'id': '101-001-29655670-001', 'alias': 'Primary', 'currency': 'USD', 'balance': '99880.484', 'createdByUserID': 29655670, 'createdTime': '2024-08-08T00:09:15.700565792Z', 'guaranteedStopLossOrderMode': 'DISABLED', 'pl': '-52.77', 'resettablePL': '-52.77', 'resettablePLTime': '0', 'financing': '-66.746', 'commission': '0.0', 'guaranteedExecutionFees': '0.0', 'marginRate': '0.02', 'openTradeCount': 1, 'openPositionCount': 1, 'pendingOrderCount': 0, 'hedgingEnabled': False, 'unrealizedPL': '-688.37', 'NAV': '99192.114', 'marginUsed': '410.2556', 'marginAvailable': '98783.9484', 'positionValue': '20512.78', 'marginCloseoutUnrealizedPL': '-686.28', 'marginCloseoutNAV': '99194.204', 'marginCloseoutMarginUsed': '410.2556', 'marginCloseoutPercent': '0.00207', 'marginCloseoutPositionValue': '20512.78', 'withdrawalLimit': '98783.9484', 'marginCallMarginUsed': '410.2556', 'marginCallPercent': '0.00414', 'lastTransactionID': '120'}

Trying parameters: {'price': 'M', 'granularity'

In [31]:
trader.position

0

In [33]:
trader.units

5000

In [35]:
trader.get_positions()

[{'instrument': 'EUR_USD',
  'pl': '-52.77',
  'unrealizedPL': '-688.37',
  'marginUsed': '410.2556',
  'resettablePL': '-52.77',
  'financing': '-66.746',
  'commission': '0.0',
  'guaranteedExecutionFees': '0.0',
  'long': {'units': '19000.0',
   'averagePrice': 1.11574,
   'tradeIDs': ['42'],
   'pl': '-16.83',
   'unrealizedPL': '-688.37',
   'resettablePL': '-16.83',
   'financing': '-66.9039',
   'guaranteedExecutionFees': '0.0'},
  'short': {'units': '0.0',
   'pl': '-35.94',
   'unrealizedPL': '0.0',
   'resettablePL': '-35.94',
   'financing': '0.1579',
   'guaranteedExecutionFees': '0.0'}}]